In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
class TreeNode:
    def __init__(self,data,output):
        self.data = data   # data represents the feature upon which the node was split when fitting the training data
        self.output = output # output represents the class with current majority at this instance of the decision tree
        self.children = {}  
        self.index = -1
    
    def add_child(self,feature_value,obj):
        self.children[feature_value] = obj

In [4]:
class DecisionTreeClassifier:
    def __init__(self):
        self.__root = None  # root represents the root node of the decision tree built after fitting the training data
    
    def __count_unique(self,Y):
        d = {}   # returns a dictionary with keys as unique values of Y and the corresponding value as its frequency
        for i in Y:
            if i not in d:
                d[i] = 1
            d[i] += 1
        return d
    
    def __entropy(self,Y):  # returns the entropy
        freq = self.__count_unique(Y)
        total = len(Y)
        entropy = 0
        for i in freq:
            p = freq[i]/total
            entropy += -p*np.log(p)
        return entropy
    
    def __gain_ratio(self,X,Y,feature):   # returns the gain ratio
        info_i = self.__entropy(Y)   # entropy before splitting
        info_f = 0    # entropy after splitting
        split_info = 0
        values = set(X[:,feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        initial_size = df.shape[0]
        for i in values:
            df1 = df[df[feature] == i]
            current_size = df1.shape[0]
            split_info += -(current_size/initial_size)*np.log(current_size/initial_size)
            info_f += (current_size/initial_size)*self.__entropy(df1[df1.shape[1]-1])
        
        if split_info == 0:  # in case split info is zero
            return math.inf
        
        info_gain = info_i - info_f
        gain_ratio = info_gain/split_info
        return gain_ratio
    
    def __gini_index(self,Y):  # returns the gini index 
        freq = self.__count_unique(Y)
        total = len(Y)
        gini_index = 1
        for i in freq:
            p = freq[i]/total
            gini_index -= p*p
        return gini_index    
    
    def __gini_gain(self,X,Y,feature):
        gini_i = self.__gini_index(Y)  # gini index before splitting
        gini_f = 0  # gini index after splitting
        values = set(X[:,feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        initial_size = df.shape[0]
        for i in values:
            df1 = df[df[feature] == i]
            current_size = df1.shape[0]
            gini_f += (current_size/initial_size)*self.__gini_index(df1[df1.shape[1] - 1])
            
        gini_gain = gini_i - gini_f
        return gini_gain
    
    def __decision_tree(self,X,Y,features,level,classes):
        
        # returns the root of the Decision Tree(which consists of TreeNodes) built after fitting the training data
        # classes represents the different classes present in the classification problem 
        # level represents depth of the tree
        
        
        # If the node consists of only 1 class
        if(len(set(Y)) == 1):
            print("Level" , level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
                    print("Count of" , i , "=",len(Y))
                else:
                    print("count of" , i , "= 0")
                
            print("current entropy is = 0.0")
            print("Reached leaf node")
            print()
            return TreeNode(None,output)
        
        # If we have run out of features to split upon
        # In this case we will output the class with maximum count
        if(len(features) == 0):
            print("Level" , level)
            output = None
            freq = self.__count_unique(Y)
            max_count = -math.inf
            for i in classes:
                if i not in freq:
                    print("count of" , i , "= 0")
                else:
                    if freq[i] > max_count:
                        max_count = freq[i]
                        output = i
                    print("count of" , i , "=" , freq[i])
                    
            print("Current entropy is =" , self.__entropy(Y))     
            print("Reached leaf node")
            print()
            return TreeNode(None,output)
        
        
        # Finding the best feature to split upon
        max_gain = -math.inf
        final_feature = None
        for f in features:
            current_gain = self.__gain_ratio(X,Y,f)
            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f
                
        print("Level" , level)
        output = None
        freq = self.__count_unique(Y)
        max_count = -math.inf
        for i in classes:
            if i not in freq:
                print("count of" , i ,"= 0")
            else:
                if freq[i] > max_count:
                    output = i
                    max_count = freq[i]
                print("count of" , i ,"=",freq[i])
                
        print("current entropy is =",self.__entropy(Y)) 
        print("Splitting on feature X[" , final_feature,"] with gain ratio ", max_gain, sep = "")
        print()
        
        unique_values = set(X[:,final_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        
        current_node = TreeNode(final_feature,output)
        
        # removing the selected feature from the list as we do not want to split on one feature more than once
        index = features.index(final_feature)
        features.remove(final_feature)
        
        for i in unique_values:
            df1  = df[df[final_feature] == i]  # Creating a new dataframe with value of selected feature i
            
            # Segregating the X and Y values and recursively calling on the splits
            node = self.__decision_tree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:,df1.shape[1]-1].values,features,level+1,classes)
            current_node.add_child(i,node) 
            
        # Add the removed feature
        features.insert(index,final_feature)
        return current_node
    
    def fit(self,X,Y):
        # Fits to the given training data
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        self.__root = self.__decision_tree(X,Y,features,level,classes)
        
    def __predict_for(self,data,node):
        # predicts the class for a given testing point and returns the answer
        
        # if We have reached a leaf node
        if len(node.children) == 0 :
            return node.output

        val = data[node.data] # represents the value of feature on which the split was made       
        if val not in node.children :
            return node.output
        
        # Recursively call on the splits
        return self.__predict_for(data,node.children[val])

    def predict(self,X):
        # This function returns Y predicted
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.__predict_for(X[i],self.__root)
        return Y  
      

In [7]:
from sklearn.tree import export_graphviz
import pydotplus
X = np.array([[1,1],[1,0],[0,1],[0,0]])
Y = [1,1,1,0]
alg = DecisionTreeClassifier()
alg.fit(X,Y)
dot_data = export_graphviz(alg , out_file = None , filled = True , rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data)


Level 0
count of 0 = 2
count of 1 = 4
current entropy is = 0.34657359027997264
Splitting on feature X[0] with gain ratio 0.9387218755408672

Level 1
count of 0 = 2
count of 1 = 2
current entropy is = 0.0
Splitting on feature X[1] with gain ratio 2.0

Level 2
Count of 0 = 1
count of 1 = 0
current entropy is = 0.0
Reached leaf node

Level 2
count of 0 = 0
Count of 1 = 1
current entropy is = 0.0
Reached leaf node

Level 1
count of 0 = 0
Count of 1 = 2
current entropy is = 0.0
Reached leaf node



NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [8]:
X[0]

array([1, 1])